In [ ]:
# Imports
MIN_REQ_PYTHON = (3,6)
import ldap3
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
import string
from tkinter import Tk
from tkinter.filedialog import askopenfilename

In [ ]:
# INPUT A DN LIST
# ATTENTION: no header
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

#f = open(filename, "r")
#print(f.read())
df_dnlist = pd.read_csv(filename,sep=';',header=None)
str_base_dn = ""

In [ ]:
# INPUT THE DC BASE TO TRIM
str_base_dn = input()

In [ ]:
print(str_base_dn)

In [ ]:
# BUILD 'df_host_dn'
d = {'host':[""], 'dn':[""]}

df_host_dn = pd.DataFrame(data=d)
index=0

for str_dn in df_dnlist[1]:
    
    str_dn_short=""

    if str_base_dn != "":
        str_dn_short = str_dn.strip().split(str_base_dn)[0]
        str_dn_short = str_dn_short.strip(",")
    else:
        str_dn_short = str_dn.strip()
    
    df_dn_cast = ldap3.utils.dn.parse_dn(str_dn_short)
    df_host_dn.at[index, 'host'] = df_dn_cast[0][1]
    
    i=0; ou =""
    for xn in df_dn_cast:
        if i > 0:
            ou = ou + xn[0]+"="+xn[1]+xn[2]
        i = i +1
    
    df_host_dn.at[index, 'dn'] = ou
    index = index + 1

display(df_host_dn)


In [ ]:
# V1.0 => PLOT THE PIE CHART
plotty = df_host_dn.groupby('dn').count()
plot = plotty.plot.pie(y='host',autopct="%i", figsize=(15,15))
plot
# filter ALB listening on ports 443 and 80 
# pie chart per HTTP return code
#ports = [80, 443]
#dff = df[df.Port.isin(ports)]
#plott = dff.groupby('HTTP return code').count()
#plot = plott.plot.pie(y='Host', autopct="%i", figsize=(5, 5))

In [ ]:
# V1.1 => SUNBURST CHART / BUILD THE DF 
int_level_max = 0
int_level_base_dn = 0

if str_base_dn != "":
    dict_base_dn = ldap3.utils.dn.parse_dn(str_base_dn)
    int_level_base_dn = len(dict_base_dn)
else:
    dict_base_dn = None

print ("Base : " + str_base_dn + "\nBase DN : " + str(dict_base_dn) + "\nNb niveaux : " + str(int_level_base_dn))

### STEP 1: Compute the number of OU levels / Initialize the list_OUs
for str_dn in df_host_dn['dn']:
  
    # get the dn + base + level_max
    dict_dn_short=""
    dict_dn_cast = ldap3.utils.dn.parse_dn(str_dn)
    int_dn_len = len(dict_dn_cast)
    if ( int_dn_len > int_level_max):
        int_level_max = int_dn_len

w, h = len(df_dnlist[1]), int_level_max
list_OUs = [[None for x in range(w)] for y in range(h)] 

print("Nb niveaux max: " + str(int_level_max))

### STEP 2: Populate df_OUs
idx=0
for str_dn in df_host_dn['dn']:
           
    dict_dn_cast = ldap3.utils.dn.parse_dn(str_dn)
    int_dn_len = len(dict_dn_cast)

    i = int_dn_len - 1
    for xn in dict_dn_cast: 
        ou = xn[0]+"="+xn[1]
        list_OUs[i][idx] = ou
        i = i - 1

    idx = idx + 1

In [ ]:
# V1.1 => PLOT THE SUNBURST CHART 
import plotly.express as px
from plotly.offline import iplot 

hits = [1 for x in range(idx)]
dff = pd.DataFrame(dict(ou0=list_OUs[0], ou1=list_OUs[1], ou2=list_OUs[2], ou3=list_OUs[3], ou4=list_OUs[4], hits=hits))
#dff = dff.dropna()
fig = px.sunburst(dff, path=['ou0','ou1', 'ou2', 'ou3', 'ou4'],values='hits')
fig.show()

In [ ]:
# EXPORT THE CHART TO SVG
import plotly.graph_objects as go
f = go.Figure( fig)
f.write_image("fig1.svg")